### Importar librerias, configuraciones y la ATT-GAN

In [1]:
import torch
import torch.nn as nn
from nn import LinearBlock, Conv2dBlock, ConvTranspose2dBlock
from torchsummary import summary
import json
import torch.utils.data as data
import h5py
import pickle as pkl
import torchvision.transforms as transforms
from jmetal.algorithm.multiobjective import NSGAII
from jmetal.operator import SBXCrossover, PolynomialMutation
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.util.termination_criterion import StoppingByQualityIndicator
from jmetal.util.observer import ProgressBarObserver
from jmetal.core.problem import FloatProblem
from jmetal.core.solution import FloatSolution
from jmetal.lab.experiment import Experiment, Job, generate_summary_from_experiment
from jmetal.core.quality_indicator import *
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from jmetal.core.quality_indicator import HyperVolume
import os
import inspect
from jmetal.util.ranking import FastNonDominatedRanking
from jmetal.util.density_estimator import CrowdingDistance
from jmetal.operator.selection import BinaryTournamentSelection
from jmetal.util.comparator import MultiComparator
from jmetal.util.comparator import DominanceWithConstraintsComparator

# custom scripts
from my_attgan import AttGAN
from my_mivolo_inference import mivolo_inference
from my_mivolo_inference import predictor
from cf_utils import *
from generate_gender_cfs import AttGanPlausibleCounterfactualProblem
from data import Custom
from data import CelebA_HQ_custom

Model summary (fused): 112 layers, 68,125,494 parameters, 0 gradients, 257.4 GFLOPs


In [2]:
# Load settings and base att names

with open('./384_shortcut1_inject1_none_hq/setting.txt', 'r') as f:
    gan_args = json.load(f)
    
base_attrs = gan_args.get('attrs')

# Load AttGAN

attgan = AttGAN(gan_args)
attgan.load('./384_shortcut1_inject1_none_hq/weights.149.pth')
attgan.eval()

### Cargar los datos para generar los CF

In [3]:
custom_image = 'from_front'
path = './Counterfactuals/Front_10374_0.pkl'

if custom_image == 'custom':

  # Load dataloader ATTGAN repo

  test_dataset = Custom('./data/custom', './data/list_attr_custom.txt', gan_args.get('img_size'), gan_args.get('attrs'))
  test_dataloader = data.DataLoader(
      test_dataset, batch_size=1, num_workers=gan_args.get('num_workers'),
      shuffle=False, drop_last=False
  )
  # Normalizes data using mean 0.5 and std 0.5 -> range [-1, 1]

  # Test Data
  
  training_set_images = torch.tensor([])
  training_set_attributes = torch.tensor([])
  training_set_file_names = []
  
  for idx, (img_a, att_a, file_nm) in enumerate(test_dataloader):
    training_set_images = torch.cat((training_set_images, img_a), dim = 0)
    training_set_attributes = torch.cat((training_set_attributes, att_a), dim = 0)
    training_set_file_names.append(file_nm)
    
  # Receives PIL Image (384, 384, 3) x (0, 255)
  # To Tensor scales to (0, 1)
  # Normalize scales to (-1, 1) [(0 - 0.5)/0.5, (1 - 0.5)/0.5]
  # Outputs tensor (1, 3, 384, 384) x (-1, 1)
  
elif custom_image == 'random':
  
  # CelebaHQ N samples

  celeba_path = './celeba_hq_dataset/CelebA-HQ-img'
  atts_path = './celeba_hq_dataset/CelebAMask-HQ-attribute-anno.txt'
  base_attrs = gan_args.get('attrs')

  sample_celeba_data = CelebA_HQ_custom(
                        data_path = celeba_path,
                        attr_path = atts_path,
                        selected_attrs = base_attrs,
                        image_size = gan_args.get('img_size'),
                        mode = 'train'
                      )

  sample_celeba_dataloader = data.DataLoader(
                              sample_celeba_data, batch_size= 1, num_workers=gan_args.get('num_workers'),
                              shuffle=True, drop_last=False
                            )
  
  data_iterator = iter(sample_celeba_dataloader)
  training_set_images, training_set_attributes, training_set_file_names = next(data_iterator)
  
elif custom_image == 'from_front':
  with open(path, 'rb') as f:
    pareto_front = pkl.load(f)
    training_set_images = pkl.load(f)
    training_set_attributes = pkl.load(f)
    training_set_file_names = [(str.split(path, '_')[1] + '.jpg', )]

### Generar los CFs

In [4]:
# Define factual image

null_intervention = False

sample_idx = 0
factual_img = torch.index_select(training_set_images, 0, torch.tensor(sample_idx))
factual_atts = torch.index_select(training_set_attributes, 0, torch.tensor(sample_idx))
img_file_name = training_set_file_names[sample_idx][0]
d_real, dc_real = attgan.D(factual_img)
factual_img_recons = attgan.G(factual_img, dc_real)
lpips = LearnedPerceptualImagePatchSimilarity(net_type='alex', reduction='none')
lpips_recons = lpips(factual_img, factual_img_recons).item()

prediction_orig, _ = mivolo_inference(factual_img, True)

if prediction_orig >= 0.5:
  desired_pred = 0
else:
  desired_pred = 1
  
if null_intervention:
  desired_pred = 1 - desired_pred
  
print(prediction_orig, desired_pred)

0.9972637891769409 0


In [6]:
%matplotlib inline

# Generate CFs

# Hyperparameters

pop_size = 100
max_evals = 5000
use_lpips = False

if use_lpips:
  original_discriminator_score = lpips_recons
else:
  original_discriminator_score = d_real.detach()

# Solver 

problem = AttGanPlausibleCounterfactualProblem(
            image = factual_img, 
            code = dc_real, # use the predicted scores for each attribute
            decoder = attgan.G, 
            discriminator = attgan.D, 
            classifier = mivolo_inference, 
            original_pred = prediction_orig,
            original_discriminator_score = original_discriminator_score,
            desired_pred = desired_pred,
            use_lpips = use_lpips,
            non_actionable_features = None
          )

algorithm = NSGAII(
             problem=problem,
             population_size=pop_size,
             offspring_population_size=pop_size,
             selection = BinaryTournamentSelection(
                           MultiComparator([DominanceWithConstraintsComparator(), CrowdingDistance.get_comparator()])
                         ),
             mutation=PolynomialMutation(
                 probability=1/problem._number_of_variables,
                 distribution_index=20),
             crossover=SBXCrossover(probability=0.9, distribution_index=20),
             termination_criterion=StoppingByEvaluations(max_evaluations=max_evals),
             dominance_comparator = DominanceWithConstraintsComparator()
         )

#jobs = [Job(algorithm, algorithm_tag = 'NSGAII', problem_tag = 'Gender_CF', run = max_evals)]
#experiment = Experiment(output_dir='./Counterfactuals', jobs=jobs)
#pareto_front = experiment.jobs[0].algorithm.result()

progress_bar = ProgressBarObserver(max = max_evals)
algorithm.observable.register(progress_bar)
hipervolume_indicator = HyperVolumeObserver(frequency=100, img_file_name = img_file_name)
algorithm.observable.register(hipervolume_indicator)
        
algorithm.run()
pareto_front = algorithm.result()
runtime_in_seconds = round(algorithm.total_computing_time, 3)

write_pkl_file(img_file_name, algorithm, problem, factual_img, factual_atts, pareto_front, runtime_in_seconds, prediction_orig, desired_pred, pop_size, max_evals)

[2025-12-11 15:52:16,211] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2025-12-11 15:52:16,212] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2025-12-11 15:53:24,968] [jmetal.core.algorithm] [DEBUG] Initializing progress...
Progress:   0%|          | 0/5000 [00:00<?, ?it/s][2025-12-11 15:53:24,975] [jmetal] [INFO] Evaluations: 100 
 HyperVolume: 0.8272536355201469 
 Computing time: 68.75938892364502
[2025-12-11 15:53:25,364] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met


AttributeError: 'Tensor' object has no attribute 'get'